<a href="https://colab.research.google.com/github/hoangvanphuong6402/Deep-learning/blob/main/trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==1.12.0+cu113 torchvision==0.13.0+cu113 torchaudio==0.12.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 727.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 58.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu121
    Uninstalling torchvision-0.20.1+cu121:
      Successfully uninstalled torchvision-0.20.1+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.5.1+cu121
    Uninstalling torchaudio-2.5.1+cu121:
      Successfully uninstalled torchaudio-2.5.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fol

In [2]:
import torch

In [3]:
!git clone https://github.com/JaidedAI/EasyOCR.git

Cloning into 'EasyOCR'...
remote: Enumerating objects: 2750, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 2750 (delta 2), reused 0 (delta 0), pack-reused 2740 (from 2)
Receiving objects: 100% (2750/2750), 157.85 MiB | 18.78 MiB/s, done.
Resolving deltas: 100% (1667/1667), done.
Updating files: 100% (313/313), done.


In [4]:
%cd EasyOCR

/content/EasyOCR


In [5]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 31.7 MB/s eta 0:00:00


In [8]:
%cd trainer

/content/EasyOCR/trainer


In [16]:
import os
import torch.backends.cudnn as cudnn
import yaml
import train
from utils import AttrDict
import pandas as pd

In [11]:
cudnn.benchmark = True
cudnn.deterministic = False

In [12]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [14]:
!unzip /content/en_sample.zip -d /content/en_sample

Archive:  /content/en_sample.zip
   creating: /content/en_sample/en_sample/
  inflating: /content/en_sample/en_sample/labels.csv  
  inflating: /content/en_sample/en_sample/2.jpg  
  inflating: /content/en_sample/en_sample/4.jpg  
  inflating: /content/en_sample/en_sample/3.jpg  
  inflating: /content/en_sample/en_sample/13.jpg  
  inflating: /content/en_sample/en_sample/1.jpg  
  inflating: /content/en_sample/en_sample/6.jpg  
  inflating: /content/en_sample/en_sample/16.jpg  
  inflating: /content/en_sample/en_sample/8.jpg  
  inflating: /content/en_sample/en_sample/17.jpg  
  inflating: /content/en_sample/en_sample/14.jpg  
  inflating: /content/en_sample/en_sample/9.jpg  
  inflating: /content/en_sample/en_sample/15.jpg  
  inflating: /content/en_sample/en_sample/7.jpg  
  inflating: /content/en_sample/en_sample/12.jpg  
  inflating: /content/en_sample/en_sample/10.jpg  
  inflating: /content/en_sample/en_sample/18.jpg  
  inflating: /content/en_sample/en_sample/22.jpg  
  inflatin

In [ ]:
opt = get_config("config_files/en_filtered_config.yaml")
train.train(opt, amp=False)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: /content/en_sample/en_sample
opt.select_data: ['/content/en_sample/en_sample']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    /content/en_sample/en_sample	 dataset: /content/en_sample/en_sample
/content/en_sample/en_sample/
sub-directory:	/.	 num samples: 882
num total samples of /content/en_sample/en_sample: 882 x 1.0 (total_data_usage_ratio) = 882
num samples of /content/en_sample/en_sample per batch: 32 x 1.0 (batch_ratio) = 32


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


--------------------------------------------------------------------------------
Total_batch_size: 32 = 32
--------------------------------------------------------------------------------
dataset_root:    /content/en_sample/en_sample	 dataset: /
/content/en_sample/en_sample/


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


sub-directory:	/.	 num samples: 882
--------------------------------------------------------------------------------
No Transformation module specified
model input parameters 64 600 20 1 256 256 97 34 None VGG BiLSTM CTC
Model:
DataParallel(
  (module): Model(
    (FeatureExtraction): VGG_FeatureExtractor(
      (ConvNet): Sequential(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): ReLU(inplace=True)
        (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (7): ReLU(inplace=True)
        (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (9): ReLU(inplace=True)
        (10): MaxPool2d(kernel_size=(2, 1